### simple APP using langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['GEMINI_API_KEY']= os.getenv('GEMINI_API_KEY')
os.environ['LANGCHAIN_API_KEY']= os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'True'
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [4]:
#load data from web
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader = WebBaseLoader("https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui")
doc = loader.load()
doc

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt Engineering Quick Start (UI) | 🦜️🛠️ LangSmith', 'description': 'This quick start will walk through how to create, test, and iterate on prompts in LangSmith.', 'language': 'en'}, page_content='\n\n\n\n\nPrompt Engineering Quick Start (UI) | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringQuickstartsQuick Start (UI)Quick Start (SDK)TutorialsOptimize a classifierSync Prompts with GitHubHow-to GuidesCreate a promptRun the playground against a custom LangServe model serverRun the playground against an OpenAI-compliant model provider/proxyUpdate a promptManage prompts programmaticallyManaging Prompt SettingsCommit TagsOpen a prompt from a tracePublic prompt hubPrompt Canva

In [6]:
#split loaded data
from langchain_text_splitters import RecursiveCharacterTextSplitter

split = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
docs = split.split_documents(doc)
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt Engineering Quick Start (UI) | 🦜️🛠️ LangSmith', 'description': 'This quick start will walk through how to create, test, and iterate on prompts in LangSmith.', 'language': 'en'}, page_content='Prompt Engineering Quick Start (UI) | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt Engineering Quick Start (UI) | 🦜️🛠️ LangSmith', 'description': 'This quick start will walk through how to create, test, and iterate on prompts in LangSmith.', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringQuickstartsQuick Start (UI)Quick Start (SDK)TutorialsOptimize a classifierSync Prompts with

In [7]:
#embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


c:\Users\User1\anaconda3\envs\langchain-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.vectorstores import FAISS
vectordb = FAISS.from_documents(docs, embedding)
vectordb

In [9]:
query = "LangSmith makes it easy to iterate on prompts with your entire team."
result = vectordb.similarity_search(query)
result[0].page_content

'4. Save a prompt\u200b\nOne you have run some tests and made your desired changes to your prompt you can click the “Save” button to save your prompt for future use.\n\n5. Iterate on a prompt\u200b\nLangSmith makes it easy to iterate on prompts with your entire team. Members of your workspace can select a prompt to iterate on in the playground,\nand once they are happy with their changes, they can simply save it as a new commit.\nTo improve your prompts:'

In [10]:
# llm model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following questions based only on the provided contect:
<context>
{context}
</context>
"""
)

doc_chain = create_stuff_documents_chain(llm, prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based only on the provided contect:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002A920C5D930>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [12]:
from langchain_core.documents import Document
doc_chain.invoke({
    "input": "LangSmith makes it easy to iterate on prompts with your entire team.",
    "context": [Document(page_content="LangSmith makes it easy to iterate on prompts with your entire team")]
})

'Please provide the questions you would like me to answer. I will base my responses solely on the text provided.'

In [13]:
vectordb

In [14]:
from langchain.chains import create_retrieval_chain
retriever = vectordb.as_retriever()
r_chain = create_retrieval_chain(retriever, doc_chain)
r_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002A920C5C4F0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based only on the provided contect:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
     

In [17]:
result2 = r_chain.invoke({"input": "LangSmith makes it easy to iterate on prompts with your entire team." })
result2

{'input': 'LangSmith makes it easy to iterate on prompts with your entire team.',
 'context': [Document(id='a0827834-6a05-4ffe-a542-1732ea6a7ed2', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt Engineering Quick Start (UI) | 🦜️🛠️ LangSmith', 'description': 'This quick start will walk through how to create, test, and iterate on prompts in LangSmith.', 'language': 'en'}, page_content='4. Save a prompt\u200b\nOne you have run some tests and made your desired changes to your prompt you can click the “Save” button to save your prompt for future use.\n\n5. Iterate on a prompt\u200b\nLangSmith makes it easy to iterate on prompts with your entire team. Members of your workspace can select a prompt to iterate on in the playground,\nand once they are happy with their changes, they can simply save it as a new commit.\nTo improve your prompts:'),
  Document(id='79619de3-bba0-4109-b9a1-3f0f8338818b', metadata={'source': 'https:/

In [18]:
result2['answer']

'Based only on the provided context, here are the answers to potential questions:\n\n**Q: How do you create a new prompt in LangSmith?**\nA: To create a prompt, you navigate to the "Prompts" section of the left-hand sidebar and click on the “+ New Prompt” button.\n\n**Q: What is the only setup required for this guide?**\nA: The only setup needed is to make sure you have signed up for a LangSmith account.\n\n**Q: How do you save a prompt?**\nA: Once you have made your desired changes to a prompt, you can click the “Save” button to save it for future use.\n\n**Q: How can team members iterate on a prompt?**\nA: Members of a workspace can select a prompt to iterate on in the playground. Once they are happy with their changes, they can save it as a new commit.\n\n**Q: What tool has LangSmith created to help with iterating on prompts?**\nA: LangSmith has created Prompt Canvas, which is an interactive tool to build and optimize prompts.\n\n**Q: Does this guide cover using the SDK for prompt e

In [19]:
result2['context']

[Document(id='a0827834-6a05-4ffe-a542-1732ea6a7ed2', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt Engineering Quick Start (UI) | 🦜️🛠️ LangSmith', 'description': 'This quick start will walk through how to create, test, and iterate on prompts in LangSmith.', 'language': 'en'}, page_content='4. Save a prompt\u200b\nOne you have run some tests and made your desired changes to your prompt you can click the “Save” button to save your prompt for future use.\n\n5. Iterate on a prompt\u200b\nLangSmith makes it easy to iterate on prompts with your entire team. Members of your workspace can select a prompt to iterate on in the playground,\nand once they are happy with their changes, they can simply save it as a new commit.\nTo improve your prompts:'),
 Document(id='79619de3-bba0-4109-b9a1-3f0f8338818b', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_ui', 'title': 'Prompt Engin

In [20]:
result2['input']

'LangSmith makes it easy to iterate on prompts with your entire team.'